In [229]:
import numpy as np
import pandas as pd
import pgeocode
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

In [230]:
# data17 = pd.read_csv("C:\\Users\\ARU PC\\Documents\\GitHub\\Data\\R13352078_SL860_2017.csv", header=[0,1])

In [231]:
df = data17.copy()

In [232]:
df.head()

Name of Area Total Population                \
      Geo_NAME    SE_A00001_001 SE_A00002_001   
0  01001 ZCTA5            17537         17537   
1  01002 ZCTA5            30280         30280   
2  01003 ZCTA5            11131         11131   
3  01005 ZCTA5             5014          5014   
4  01007 ZCTA5            14906         14906   

  Population Density (Per Sq. Mile)   Area (Land)   Area Total:  \
                      SE_A00002_002 SE_A00002_003 SE_A00003_001   
0                         1532.6440     11.442319     12.303240   
1                          550.1321     55.041324     56.693940   
2                        15647.7500      0.711348      0.716286   
3                          113.3306     44.242253     44.499800   
4                          283.1499     52.643487     55.324520   

  Area Total: Area (Land) Area Total: Area (Water) Total Population:  \
            SE_A00003_002            SE_A00003_003     SE_A02001_001   
0               11.442320                 0.860919             17537   
1               55.041320                 1.652618             30280   
2                0.711349                 0.004937             11131   
3               44.242250                 0.257546              5014   
4               52.643490                 2.681037             14906   

  Total Population: Male  ... Own Children under 18 Years  \
           SE_A02001_002  ...               SE_A10065_001   
0                   8211  ...                        2706   
1                  14881  ...                        3625   
2                   5686  ...                           0   
3                   2581  ...                         813   
4                   7265  ...                        3153   

  Own Children under 18 Years: Children Living with Single Parents  \
                                                     SE_A10065_002   
0                                                627                 
1                                               1289                 
2                                                  0                 
3                                                184                 
4                                                553                 

     Households Households: 1-Person Household Households: 2-Person Household  \
  SE_A10066_001                  SE_A10066_002                  SE_A10066_003   
0          7460                           2826                           2460   
1          9976                           2726                           3185   
2            31                             18                              8   
3          1785                            503                            534   
4          5558                           1022                           2097   

  Households: 3-Person Household Households: 4-Person Household  \
                   SE_A10066_004                  SE_A10066_005   
0                           1010                            785   
1                           1632                           1800   
2                              0                              5   
3                            322                            232   
4                            854                           1143   

  Households: 5-Person Household Households: 6-Person Household  \
                   SE_A10066_006                  SE_A10066_007   
0                            237                             83   
1                            455                            103   
2                              0                              0   
3                            119                             35   
4                            364                             78   

  Households: 7-or-More Person Household  
                           SE_A10066_008  
0                                     59  
1                                     75  
2                                      0  
3                                     40 

# Preprocess Data


## Extract zip codes


In [233]:
df.insert(0, 'zip_code', df[('Name of Area', 'Geo_NAME')].astype(str).str[:5])
df.drop('Name of Area', level = 0, axis =1, inplace=True)
df = df.loc[pd.to_numeric(df.zip_code).between(10000,20000)] # Filtering rows to limit the data we work on. Selecting group of states starting with 1

In [234]:
df.set_index('zip_code', inplace=True)

In [235]:
df.shape

(3658, 2177)

### Extracting only the tables required for the analysis from the dataset


In [236]:
table_names = ['SE_B13004', 'SE_B18002', 'SE_A03001', 'SE_A13007', 'SE_A14028', 'SE_A14024_', 'SE_B12001', 'SE_A12003', 'SE_A12004', 'SE_A17005', 'SE_A17009', 'SE_A20001', 'SE_A00002']
matching_columns = [col for col in df.columns.get_level_values(1) if col.startswith(tuple(table_names))]
df = df.loc[:, df.columns.get_level_values(1).isin(matching_columns)]

In [237]:
# Check for mimssing values
df = df.dropna() # Drop rows with missing values

## Processing Y variable

In [238]:
df['underPoverty'] = np.where(df[('Population for Whom Poverty Status Is Determined: 2.00 and Over (Doing Ok)', 'SE_B13004_005')].div(df[('Population for Whom Poverty Status Is Determined:', 'SE_B13004_001')]) < 0.5,1,0) 
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_B13004')] # Drop the columns from the table after processing


In [239]:
perCapita = ((df.loc[:,df.columns.get_level_values(1).str.startswith('SE_A14024_001')]))

In [240]:
perCapita['Per Capita Income (In 2017 Inflation Adjusted Dollars)']

,SE_A14024_001
zip_code,
10001,86014.0
10002,30677.0
10003,91141.0
10004,105390.0
10005,102957.0
...,...
19971,56647.0
19973,26878.0
19975,33055.0


In [241]:
# #INCOME
df['incomeBelowPovertyLevelMajority'] = np.where((df[('Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months Below Poverty Level', 'SE_A13007_002')].values) /(df[('Civilian Population 16 Years and Over for Whom Poverty Status Is Determined', 'SE_A13007_001')])>0.5,1,0)
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A13007')] # Drop the columns from the table after processing

df.loc[:, 'incomePerCapita'] = df.loc[: ,df.columns.get_level_values(1).str.startswith('SE_A14024_001')]
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A14024')] # Drop the columns from the table after processing
df.loc[:, 'incomePerCapita'] = pd.qcut(df.incomePerCapita,4, labels = ['incomePerCapitaQ1','incomePerCapitaQ2','incomePerCapitaQ3','incomePerCapitaQ4'])


df.loc[:, 'incomeGiniIndex'] = df.loc[: ,df.columns.get_level_values(1).str.startswith('SE_A14028_001')] # Drop the columns from the table after processing
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A14028')] # Drop the columns from the table after processing
df.loc[:, 'incomeGiniIndex'] = pd.qcut(df.incomeGiniIndex,4, labels = ['incomeGiniIndexQ1','incomeGiniIndexQ2','incomeGiniIndexQ3','incomeGiniIndexQ4'])
# ##################################

#Race, Population 
df.loc[:, 'popDensity'] = df.loc[:,('Population Density (Per Sq. Mile)','SE_A00002_002')].values 
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A00001')] # Drop the columns from the table after processing
df.loc[:, 'popDensity'] = pd.qcut(df.popDensity,4, labels = ['popDensityQ1','popDensityQ2','popDensityQ3','popDensityQ4'])

df.drop(['SE_A00002_001','SE_A00002_003', ], axis = 1, level = 1, inplace = True)

df.drop(df.loc[df.loc[:,df.columns.get_level_values(1).str.startswith('SE_A03001_001')].isin([0]).any(axis = 1)].index, inplace = True)


df.loc[:, 'whiteMajority'] = np.where(((df.loc[:,'Total Population: White Alone'].values) / (df.loc[:,'Total Population:'].values))>0.5,1,0) # If the Majority of white alone population is more than 50% we set this binary variable to 1
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A03001')] # Drop the columns from the table after processing
# ###################################

#Employment

df = df.loc[df['Population 25 Years and Over','SE_B12001_001'] != 0] # Some of the zip codes have 0. To avoid RuntimeWarning we drop these rows

df.loc[:, 'eduHsOrLessMajority'] = np.where(((df.loc[:,('Population 25 Years and Over: Less than High School','SE_B12001_002')].values) / (df.loc[:,('Population 25 Years and Over','SE_B12001_001')].values))>0.5,1,0) # Majority of 25yo and over with less than high school education
df.loc[:, 'eduHsMajority'] = np.where(((df.loc[:,('Population 25 Years and Over: High School Diploma','SE_B12001_003')].values) / (df.loc[:,('Population 25 Years and Over','SE_B12001_001')].values))>0.5,1,0) # Majority of 25yo and over with less than high school education
df.loc[:, 'eduBachOrBetterMajority'] = np.where(((df.loc[:,('Population 25 Years and Over: Bachelor\'s Degree or Better','SE_B12001_004')].values) / (df.loc[:,('Population 25 Years and Over','SE_B12001_001')].values))>0.5,1,0) # Majority of 25yo and over with less than high school education
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_B12001_')] # Drop the columns from the table after processing

df.loc[:, 'eduEnrollmentMajority'] = np.where(((df.loc[:,('Population 3 Years and Over: Enrolled in School','SE_A12004_002')].values) / (df.loc[:,('Population 3 Years and Over:','SE_A12004_001')].values))>0.5,1,0) # If the Majority of 3yo and over enrolled in school is more than 50% we set this binary variable to 1
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A12004')] # Drop the columns from the table after processing

df.loc[:, 'eduDropoutMajority'] = np.where(((df.loc[:,('Civilian Population 16 to 19 Years: Not High School Graduate, Not Enrolled (Dropped Out)','SE_A12003_002')].values) / (df.loc[:,('Civilian Population 16 to 19 Years:','SE_A12003_001')].values))>0.5,1,0) # If the Majority of late teens dropped out from school is more than 50% we set this binary variable to 1
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A12003')] # Drop the columns from the table after processing

df.loc[:, 'employedMajority'] = np.where(((df.loc[:,('Civilian Population in Labor Force 16 Years and Over: Employed' ,'SE_A17005_002')].values) / (df.loc[:,('Civilian Population in Labor Force 16 Years and Over:','SE_A17005_001')].values))>0.5,1,0)
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A17005')] # Drop the columns from the table after processing

df.loc[:, 'employedPvtSecMajority'] = np.where(((df.loc[:,('Employed Civilian Population 16 Years and Over: Private Sector' ,'SE_A17009_002')].values) / (df.loc[:,('Employed Civilian Population 16 Years and Over:','SE_A17009_001')].values))>0.5,1,0)
df.loc[:, 'employedPubSecMajority'] = np.where(((df.loc[:,('Employed Civilian Population 16 Years and Over: Public Sector' ,'SE_A17009_003')].values) / (df.loc[:,('Employed Civilian Population 16 Years and Over:','SE_A17009_001')].values))>0.5,1,0)
df.loc[:, 'employedSelfEmpMajority'] = np.where(((df.loc[:,('Employed Civilian Population 16 Years and Over: Self-Employed (Incorporated and Not Incorporated)' ,'SE_A17009_004')].values) / (df.loc[:,('Employed Civilian Population 16 Years and Over:','SE_A17009_001')].values))>0.5,1,0)
df.loc[:, 'employedPvtNonProMajority'] = np.where(((df.loc[:,('Employed Civilian Population 16 Years and Over: Private Non-Profit' ,'SE_A17009_005')].values) / (df.loc[:,('Employed Civilian Population 16 Years and Over:','SE_A17009_001')].values))>0.5,1,0)
df.loc[:, 'employedUnpaidFamMajority'] = np.where(((df.loc[:,('Employed Civilian Population 16 Years and Over: Unpaid Family Workers' ,'SE_A17009_006')].values) / (df.loc[:,('Employed Civilian Population 16 Years and Over:','SE_A17009_001')].values))>0.001728,1,0)  #75 percentile
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A17009')] # Drop the columns from the table after processing
# # ##############################################

# #Health


df = df.loc[df.loc[:,('Total:','SE_A20001_001')] != 0] # Some of the zip codes have 0. To avoid RuntimeWarning we drop these rows
df.loc[:, 'insuredMajority'] = np.where(((df.loc[:,('Total: with Health Insurance Coverage' ,'SE_A20001_003')].values) / (df.loc[:,('Total:','SE_A20001_001')].values))>0.5,1,0)
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A20001')] # Drop the columns from the table after processing


# ##############################################

# #Housing
df = df.loc[df.loc[:, ('Renter-Occupied Housing Units', 'SE_B18002_001')] != 0] # Remove observations with 0 to avoid error while dividing
df.loc[:, 'housingRentMoreThan30pcMajority'] = np.where(((df.loc[:,('Renter-Occupied Housing Units: 30 to 49 Percent' ,'SE_B18002_002')].values + df.loc[:,('Renter-Occupied Housing Units: 50 percent or More' ,'SE_B18002_003')].values ) / (df.loc[:,('Renter-Occupied Housing Units','SE_B18002_001')].values))>0.5,1,0)
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_B18002')] # Drop the columns from the table after processing

C:\Users\ARU PC\AppData\Local\Temp\ipykernel_14028\2720873627.py:41: RuntimeWarning: invalid value encountered in true_divide
  df.loc[:, 'eduDropoutMajority'] = np.where(((df.loc[:,('Civilian Population 16 to 19 Years: Not High School Graduate, Not Enrolled (Dropped Out)','SE_A12003_002')].values) / (df.loc[:,('Civilian Population 16 to 19 Years:','SE_A12003_001')].values))>0.5,1,0) # If the Majority of late teens dropped out from school is more than 50% we set this binary variable to 1
C:\Users\ARU PC\AppData\Local\Temp\ipykernel_14028\2720873627.py:44: RuntimeWarning: invalid value encountered in true_divide
  df.loc[:, 'employedMajority'] = np.where(((df.loc[:,('Civilian Population in Labor Force 16 Years and Over: Employed' ,'SE_A17005_002')].values) / (df.loc[:,('Civilian Population in Labor Force 16 Years and Over:','SE_A17005_001')].values))>0.5,1,0)
C:\Users\ARU PC\AppData\Local\Temp\ipykernel_14028\2720873627.py:47: RuntimeWarning: invalid value encountered in true_divide
  d

In [242]:
df.to_csv("ARminingProcessing.csv")
